<a href="https://colab.research.google.com/github/Tbot101/CIS-5190-Project/blob/main/Logistic_Regression_and_FFNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


%cd /content/drive/Shareddrives/CIS\ 519\ Final\ Project

/content/drive/Shareddrives/CIS 519 Final Project


In [ ]:
%ls
# 'Reviews.csv' should be listed here

'CIS 519 Project.ipynb'       'Project Milestone 2.gdoc'
 cleaned_reviews.csv          'Project Milestone 3.docx'
 fine-tuned-bert               res/
 James_CIS_519_Project.ipynb   Reviews.csv
 logs4/                       'Taha CIS 519 Project'
'Project 1 Milestone.gdoc'    'Time-Shift CIS 519 Project'


In [ ]:
import pandas as pd
import random
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

random.seed(42)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
USE_CLEANED = True

In [ ]:
# we need to use these parameters to read in some funky text
if USE_CLEANED:
  df = pd.read_csv('cleaned_reviews.csv', engine='python', on_bad_lines='skip')
else:
  df = pd.read_csv('Reviews.csv', engine='python', on_bad_lines='skip')
  df['sentiment'] = df.apply(lambda x: 1 if x['Score'] > 3 else 0, axis=1)
  df = df.sample(frac=0.2).reset_index(drop=True)
  def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    lemma = WordNetLemmatizer()
    result = [i for i in tokens if not i in stop_words]
    lemmatized_result = []
    for word in result:
      lemmatized_result.append(lemma.lemmatize(word))

    return " ".join(lemmatized_result)
  df['cleaned_text'] = df.apply(lambda x: preprocess_text(x['Text']), axis=1)

In [ ]:
df = df.dropna()

In [ ]:
df

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment,cleaned_text
0,0,119814,B0009YUEG2,A2F09EWKV3MTO2,StillWaters,4,4,5,1303430400,This is all I ever use anymore,I love this litter because...<br />1. It compl...,1,love litter br 1 completely eliminates urine o...
1,1,290509,B004777F3M,A2PDNBEIQYRCT2,lmac60,0,0,4,1309996800,Good Tea!,Great flavor. Just enough spice. A love Chai...,1,great flavor enough spice love chai tea latte ...
2,2,489194,B002N2R7WM,A1CHGDG3NUDA5R,"C. Seggelin ""Plastered Dragon""",0,0,1,1350086400,Tasteless Fat For Your Popcorn! Just Use Butter.,"Y'know, you'd think with the words ""Buttery Fl...",0,know think word buttery flavor product name pr...
3,3,65205,B001D0GV5O,A3GIKK6DQEKEQK,CHERYL A. CATANZANO,0,0,1,1296777600,Very bland,I bought this tea because I like Timothy's Cof...,0,bought tea like timothy coffee product figured...
4,4,526585,B007PA30TG,A2XQSJG8YR6ED2,Cecilia f.,0,0,5,1303084800,great brew,i've been using these k-cups for months now. ...,1,using k cup month breakfast blend perfect peop...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113686,113686,16241,B007TJGZ54,A1PLASEQUSPDPM,Chris,0,0,5,1339718400,Price is Right,Love the Green Mountain Coffee; tastes great i...,1,love green mountain coffee taste great keurig ...
113687,113687,511398,B001E6J07I,A233H2J16V56W8,"Claxon,Bill",0,0,5,1237507200,Tasty Carrs cracker,A super glamourized cheez-it without the trans...,1,super glamourized cheez without trans fat chol...
113688,113688,211139,B000X67P2C,A3G0H0GKJ0722I,C. Smith,10,13,1,1277164800,Did not like...,I had heard so many good things about using th...,0,heard many good thing using place wheat white ...
113689,113689,520860,B0009XSXZM,A3FTHCRY3EZNA7,categ3893,1,1,5,1333843200,8 month Golden Retriever is OBSESSED.,My adolescent golden retriever just loves thes...,1,adolescent golden retriever love soon see hold...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df)

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
baseline = DummyClassifier(strategy="most_frequent")

In [ ]:
baseline.fit(train.drop('sentiment', axis=1), train['sentiment'])

DummyClassifier(strategy='most_frequent')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, log_loss

In [ ]:
baseline_predictions = baseline.predict(test.drop('sentiment', axis=1))
baseline_accuracy = accuracy_score(test['sentiment'], baseline_predictions)

In [ ]:
baseline_f1score = f1_score(test['sentiment'], baseline_predictions)

In [ ]:
def get_metrics(predictions, actual):
  accuracy = accuracy_score(actual, predictions)
  f1 = f1_score(actual, predictions)
  cross_entropy = log_loss(actual, predictions)
  print(f"Accuracy: {accuracy:.2f}")
  print(f"F1 Score: {f1:.2f}")
  print(f"Cross-Entropy: {cross_entropy:.2f}")

In [ ]:
print("-- Baseline metrics --")
get_metrics(baseline_predictions, test['sentiment'])

-- Baseline metrics --
Accuracy: 0.78
F1 Score: 0.88
Cross-Entropy: 7.89


# Logistic Regression

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

seed = 42

def model_text(X, y, model, clf_model, desc):

  X_c = model.fit_transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=seed)
  clf = clf_model.fit(X_train, y_train)

  # test accuracy
  clf_accuracy = clf.score(X_test, y_test)

  # f1 score
  preds = clf.predict(X_test)
  f1 = f1_score(y_test, preds)

  # cross entropy
  cross_entropy = log_loss(y_test, preds)

  print(f'MODEL: {desc}')
  print(f'test accuracy: {clf_accuracy:.2f}')
  print(f'f1 score: {f1:.2f}')
  print(f"Cross-Entropy: {cross_entropy:.2f}")
  print("\n")

  return (clf_accuracy, f1)


In [ ]:
desc = 'Logistic Regression (No regulaization, CountVectorizer)\n'

c = CountVectorizer()

X = df['cleaned_text']
y = df['sentiment'] # if score > 3 then sentiment=1, 0 otherwise

clf_model = LogisticRegression(penalty=None)

model_text(X, y, c, clf_model, desc)

MODEL: Logistic Regression (No regulaization, CountVectorizer)

test accuracy: 0.88
f1 score: 0.92
Cross-Entropy: 4.29




/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8811048557353977, 0.9248927515614929)

In [ ]:
desc = 'Logistic Regression (L2 Regularization, CountVectorizer)\n'

clf_model = LogisticRegression(penalty='l2')
model_text(X, y, c, clf_model, desc)

MODEL: Logistic Regression (L2 Regularization, CountVectorizer)

test accuracy: 0.89
f1 score: 0.93
Cross-Entropy: 4.10




/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.886347642505278, 0.9288984767103989)

In [ ]:
desc = 'Logistic Regression (L2 Regularization, TFIDF + n-gram)\n'

tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')
clf_model = LogisticRegression()
model_text(X, y, tfidf_n, clf_model, desc)

MODEL: Logistic Regression (L2 Regularization, TFIDF + n-gram)

test accuracy: 0.88
f1 score: 0.93
Cross-Entropy: 4.22




/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.882899366643209, 0.9290314325926559)

In [ ]:
# dataset shift, make positive-negative a 50-50 split

def shift_negative_data(test_data):
  negative_data = test_data[test_data['sentiment'] == 0]
  negative_count = len(negative_data)
  positive_data = test_data[test_data['sentiment'] == 1]
  positive_sample = positive_data.sample(n=negative_count)
  return positive_sample.append(negative_data)

desc = 'Logistic Regression (L2 Regularization, CountVectorizer, dataset shift)\n'

c = CountVectorizer()

X_c = df[['cleaned_text']]
y = df['sentiment'] # if score > 3 then sentiment=1, 0 otherwise

test_data = X_c.copy()
test_data['sentiment'] = y

test_data

test_data = shift_negative_data(test_data)

X_c_new = test_data['cleaned_text']
y_new = test_data['sentiment']

clf_model = LogisticRegression()

model_text(X_c_new, y_new, c, clf_model, desc)

<ipython-input-12-39bdca20771e>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return positive_sample.append(negative_data)


MODEL: Logistic Regression (L2 Regularization, CountVectorizer, dataset shift)

test accuracy: 0.84
f1 score: 0.84
Cross-Entropy: 5.82




/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8386063275931117, 0.8400412796697627)

# Feedforward Neural Network

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np

In [ ]:
class FFNN(nn.Module):

  def __init__(self, input_dim=40001, output_dim=2):
    super(FFNN, self).__init__()
    self.linear1 = nn.Linear(input_dim, 256) # 512
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(256, 32) # 128
    self.dropout = nn.Dropout(p=0.7) # 0.8
    self.dropout2 = nn.Dropout(p=0.7)
    self.linear3 = nn.Linear(32, output_dim)

  def forward(self, x):
    x = x.to(torch.float32)
    x = self.linear1(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.linear2(x)
    x = self.relu(x)
    x = self.dropout2(x)

    output = self.linear3(x)
    return output

In [ ]:
# tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')
df_sample = df.sample(frac=0.8)
X_c = df_sample['cleaned_text']
cv = CountVectorizer()

vec_X = cv.fit_transform(X_c)
y = np.array(df_sample['sentiment'].tolist())# if score > 3 then sentiment=1, 0 otherwise
vec_X_train, vec_X_test, y_train, y_test = train_test_split(vec_X, y, test_size=0.15)

In [ ]:
df['sentiment'].describe()

count    113678.000000
mean          0.780344
std           0.414015
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: sentiment, dtype: float64

In [ ]:
model = FFNN(vec_X_train.shape[1])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
import numpy as np
from tqdm import tqdm

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 8
batch_size = 256
num_batches = vec_X_train.shape[0] // batch_size

prev_accuracy = 0
for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.
    correct, total = 0, 0 

    for i in tqdm(range(num_batches)):
        start = i * batch_size
        end = (i + 1) * batch_size
        text = torch.tensor(vec_X_train[start:end].toarray())
        label = torch.tensor(y_train[start:end]).cuda()

        # 1. Store the inputs and labels in the GPU
        text = text.to(device)
        label = label.to(device)
        # 2. Get the model predictions
        predictions = model(text)

        # 3. Zero the gradients out
        optimizer.zero_grad()
        # 4. Get the loss
        loss = criterion(predictions, label)

        # 5. Calculate the gradients
        loss.backward()

        # 6. Update the weights
        optimizer.step()
            
        running_loss += loss.item()
        
        _, predicted = torch.max(predictions, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    curr_accuracy = float(correct) / float(total)
    print(f"Accuracy {curr_accuracy}")
    print(f"Loss: {running_loss / total}")

    if abs(curr_accuracy - prev_accuracy) < 0.005:
      print("Breaking early due to converging accuracy")
      break
    prev_accuracy = curr_accuracy
    running_loss = 0.0
    
        

100%|██████████| 301/301 [00:39<00:00,  7.56it/s]


Accuracy 0.8388314991694352
Loss: 0.001498415106033405


100%|██████████| 301/301 [00:42<00:00,  7.11it/s]


Accuracy 0.8944793397009967
Loss: 0.0010798531665661654


100%|██████████| 301/301 [00:37<00:00,  8.01it/s]


...

Accuracy 0.949296615448505
Loss: 0.0005681488369599269


100%|██████████| 301/301 [00:38<00:00,  7.80it/s]


Accuracy 0.9580694559800664
Loss: 0.00048758583983593616


100%|██████████| 301/301 [00:38<00:00,  7.89it/s]

Accuracy 0.9625207641196013
Loss: 0.0004314224217729055
Breaking early due to converging accuracy


In [ ]:
model.eval()

FFNN(
  (linear1): Linear(in_features=49368, out_features=256, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=256, out_features=32, bias=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (dropout2): Dropout(p=0.7, inplace=False)
  (linear3): Linear(in_features=32, out_features=2, bias=True)
)

In [ ]:
test_predictions = model(torch.tensor(vec_X_test.toarray()).to(device))

In [ ]:
test_predictions = test_predictions.cpu().detach().numpy()

In [ ]:
pred = np.argmax(test_predictions, axis=1)

In [ ]:
get_metrics(pred, y_test)

Accuracy: 0.90
F1 Score: 0.94
Cross-Entropy: 3.61


In [ ]:
y_test.sum()

10668

In [ ]:
len(y_test)

13642

In [ ]:
num_zeros = (y_test == 0).sum()

In [ ]:
num_zeros

2974

In [ ]:
# balance data
num_ones = 0
indexes_to_keep = []
for i, val in enumerate(y_test):
  if val == 0:
    indexes_to_keep.append(i)
  elif num_ones < num_zeros:
    num_ones += 1
    indexes_to_keep.append(i)

In [ ]:
shifted_vec_X_test = vec_X_test[indexes_to_keep]
shifted_y_test = y_test[indexes_to_keep]

In [ ]:
test_predictions = model(torch.tensor(shifted_vec_X_test.toarray()).to(device))

In [ ]:
test_predictions = test_predictions.cpu().detach().numpy()
pred = np.argmax(test_predictions, axis=1)
get_metrics(pred, shifted_y_test)

Accuracy: 0.83
F1 Score: 0.85
Cross-Entropy: 6.02
